In [1]:
import aiohttp
import asyncio
import time
import json
import pandas as pd
from bs4 import BeautifulSoup
import lxml

In [87]:
async def get_item_data(session, url):
    async with session.get(url) as resp:
        site_data = await resp.read()
        return site_data


async def main():
    
    with open(f'otomototabela.csv', 'w') as f:
        column_labels='auto'+',model'+',cena'+',rok_produkcji'+',przebieg'+',poj_silnika'+',rodzaj_paliwa'+'\n'
        f.writelines(column_labels)
    a=0
    ranges=[(range(a,a+100)),(range(a+100,a+200)),(range(a+200,a+300)),(range(a+300,a+400)),(range(a+400,a+500))]
    a+=100
    for seq in ranges:

        async with aiohttp.ClientSession() as session:
            tasks = []
            for number in seq:
                url = f'https://www.otomoto.pl/osobowe?page={number}'
                #print(url)
                tasks.append(asyncio.ensure_future(get_item_data(session, url)))
            oto_moto_page = await asyncio.gather(*tasks)



            with open(f'otomototabela.csv', 'a') as f:
                for page in oto_moto_page:
                    page=BeautifulSoup(page,features='html5')
                    for article in page.find_all('article'):
                        try:
                            f.writelines(article.h2.a.text+"|"+article.div.p.text+"|"+article.find_all('div')[5].div.span.text+"|"+article.div.ul.find_all('li')[0].text+"|"+article.div.ul.find_all('li')[1].text+"|"+article.div.ul.find_all('li')[2].text+"|"+article.div.ul.find_all('li')[3].text+'\n')
                        except:
                            continue
        print(a,'pages completed')

start_time=time.time()
await main()
print("--- %s seconds ---" % (time.time() - start_time))

100 pages completed
100 pages completed
100 pages completed
100 pages completed
100 pages completed
--- 69.17419004440308 seconds ---
